In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
# from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv', index_col=0, encoding='utf_8_sig')
# df.fillna('', inplace=True)
# label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
# df = df.rename(columns={'labels':'label','msgs':'message','diffs':'diff'})
# df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

,repo,commit,labels,msgs,diffs,feature
user,,,,,,
ponsonio,RxJava,0531b8bff5c14d9504beefb4ad47f473e3a22932,Perfective,Change hasException to hasThrowable--,diff --git a/rxjava-core/src/main/java/rx/Noti...,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0950c46beda335819928585f1262dfe1dca78a0b,Adaptive,Trying to extend the Scheduler interface accor...,diff --git a/rxjava-core/src/main/java/rx/Sche...,"[2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0f92fdd8e6422d5b79c610a7fd8409d222315a49,Adaptive,RunAsync method for outputting multiple values--,diff --git a/rxjava-contrib/rxjava-async-util/...,"[2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ..."
ponsonio,RxJava,100f571c9a2835d5a30a55374b9be74c147e031f,Corrective,forEach with Action1 but not Observer--I re-re...,diff --git a/language-adaptors/rxjava-groovy/s...,"[1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,..."
ponsonio,RxJava,191f023cf5253ea90647bc091dcaf55ccdce81cc,Corrective,1.x: Fix Completable swallows- OnErrorNotImple...,diff --git a/src/main/java/rx/Completable.java...,"[1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...
jenkinsci,clearcase-plugin,51e9da224f80254476a7dc446bca817b505381d8,Perfective,Use a temporary file to decrease memory consum...,diff --git a/src/main/java/hudson/plugins/clea...,"[2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
jexp,batch-import,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,Perfective,added a more memory efficient structure for st...,diff --git a/src/main/java/org/neo4j/batchimpo...,"[10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,..."
hdiv,hdiv,19b650c78a1c76f4fd90274d7f163f863c0d39e4,Perfective,Memory and performance optimizations,diff --git a/hdiv-config/src/main/java/org/hdi...,"[31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17..."


In [4]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [5]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/eval.csv',index=False)

In [6]:
len(train)

1246

In [7]:
train['labels'].value_counts()

labels
Corrective    424
Adaptive      420
Perfective    402
Name: count, dtype: int64

In [8]:
test['labels'].value_counts()

labels
Perfective    100
Corrective     89
Adaptive       79
Name: count, dtype: int64

In [9]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [11]:
train['embeddings'] = train['diffs'].apply(model.encode)
test['embeddings'] = test['diffs'].apply(model.encode)

In [12]:
test

,repo,commit,labels,msgs,diffs,feature,embeddings
user,,,,,,,
apache,hadoop,06d24042b64d6fa0e179b5845990068f849d9ce5,Corrective,YARN-1185. Fixed FileSystemRMStateStore to not...,diff --git a/hadoop-yarn-project/CHANGES.txt b...,"[0, 0, 0, 0, 0, 0, 0, 5, 156, 86, 5, 44, 0, 0,...","[-0.08382036, 0.15308714, -0.02434697, -0.0046..."
apache,hbase,b246e68435e8d86a2a12fb8fe6c5aa9d9896ff92,Perfective,HBASE-5259 Normalize the RegionLocation in- Ta...,diff --git a/src/main/java/org/apache/hadoop/h...,"[1, 30, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-0.08307925, 0.25729173, -0.051250797, -0.028..."
GNOME,valadoc,ca2ba9131b10c5927172505f89f79cc0088dca3f,Adaptive,libvaladoc: gir-reader: improve short descs\n,diff --git a/src/libvaladoc/content/blockconte...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04905106, 0.27592987, -0.018844165, 0.1760..."
arquillian,arquillian-graphene,2ce3373d1db087a001b226bccfba43285246a1d4,Adaptive,"chrome support added, when the browser is *chr...",diff --git a/api/src/main/java/org/jboss/arqui...,"[6, 28, 13, 1, 20, 27, 1, 0, 0, 0, 0, 0, 0, 0,...","[-0.18833342, 0.29722103, -0.033946075, -0.008..."
apache,hadoop,a3ecc3b910aa3bbc3ede2b8ba1bd040d02a26ca8,Adaptive,HDFS-4733. Make HttpFS username pattern- confi...,diff --git a/hadoop-hdfs-project/hadoop-hdfs-h...,"[3, 12, 0, 6, 0, 0, 0, 2, 76, 4, 3, 17, 0, 0, ...","[-0.08904812, 0.32778418, 0.0032636696, 0.0013..."
...,...,...,...,...,...,...,...
kiegroup,drools,8b9ffeb5f2883e3d5187ad2f4a3a35540ff70326,Corrective,JBRULES-527: fixing compilation problems--git-...,diff --git a/drools-core/src/main/java/org/dro...,"[3, 28, 6, 15, 0, 50, 2, 0, 0, 0, 0, 0, 0, 0, ...","[0.033119015, 0.2668087, 0.033190466, -0.02506..."
apache,hadoop,ed77c8925d7126f9ea3c8d9cbb1e246ad61ce37c,Corrective,YARN-596. Use scheduling policies throughout t...,diff --git a/hadoop-yarn-project/CHANGES.txt b...,"[11, 131, 38, 16, 28, 2, 8, 3, 35, 29, 42, 6, ...","[-0.06559335, 0.37437052, -0.06740786, -0.0832..."
apache,deltaspike,4600b51887de35647220ca47fa1a09042db2cb52,Corrective,DELTASPIKE-286 Comparators should be Serializa...,diff --git a/deltaspike/core/api/src/main/java...,"[1, 6, 7, 10, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...","[-0.049287796, 0.23882662, -0.024124734, 0.026..."


In [14]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['labels'].to_list()
y_test = test['labels'].to_list()

In [15]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [16]:
from sklearn import metrics
predicted = LR.predict(X_test)

print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.5649654569240212
Logistic Regression Recall: 0.5634328358208955
Logistic Regression Recall: 0.5597263394332568
Logistic Regression Accuracy: 0.5634328358208955
